In [ ]:
# Read in Microstructure Data

In [1]:
import scipy.io
import os
import numpy as np
import pandas 
import h5py
import dask.dataframe as dd
import glob
from dask import delayed
import dask

In [2]:
# dask.config.set(scheduler='single-threaded')

In [3]:
def get_files(path_):
    return sorted(glob.glob(path_))

@delayed
def read_ydata(file_):
    data = scipy.io.loadmat(file_)
    return pandas.DataFrame(data['Ceff'])

@delayed
def read_xdata(file_):
    f = h5py.File(file_, 'r')
    return pandas.DataFrame(np.array(f['M']))

def get_dfs(path_):
    return list(map(read_xdata, get_files(path_)))

yfile_path = '/home/berkay/Desktop/Graspi/50_contrast_elastic_51_51_51_homogenization/effective_C11/*.mat'
xfile_path = '/home/berkay/Desktop/Graspi/50_contrast_elastic_51_51_51_homogenization/microstructures/*.mat'

dfs = get_dfs(xfile_path)

## Here we use from_delayed so that chunks do not have to be the same size
## Each mat file does not necessarily have the same
## number of samples

dask_df = dd.from_delayed(dfs, meta=dfs[0].compute())

arr = dask_df.to_dask_array(lengths=True)
#dask_df.to_csv('export-*.csv')
#dask_df.to_parquet('microstructure.parquet', engine="pyarrow")

In [7]:
arr = arr.rechunk(chunks=(100,132651))

In [8]:
arr

,Array,Chunk
Bytes,9.44 GB,106.12 MB
Shape,"(8900, 132651)","(100, 132651)"
Count,506 Tasks,89 Chunks
Type,float64,numpy.ndarray


In [9]:
arr.to_zarr('output.zarr')